In [ ]:
!pip install datasets -q
!pip install tokenizers -q
!pip install transformers -q
!pip install seqeval -q


     |████████████████████████████████| 441 kB 8.9 MB/s 
     |████████████████████████████████| 212 kB 66.8 MB/s 
     |████████████████████████████████| 163 kB 64.7 MB/s 
     |████████████████████████████████| 115 kB 62.5 MB/s 
     |████████████████████████████████| 127 kB 51.0 MB/s 
     |████████████████████████████████| 7.6 MB 6.6 MB/s 
     |████████████████████████████████| 5.3 MB 7.5 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 


In [ ]:
from datasets import load_dataset
dataset = load_dataset("wikiann" ,"en")

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Dataset wikiann downloaded and prepared to /root/.cache/huggingface/datasets/wikiann/en/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset["train"].features["ner_tags"]
dataset["train"].features["tokens"]
dataset["train"].features["langs"]
dataset["train"].features["spans"]

Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)

In [ ]:
dataset["train"][:2]

{'tokens': [['R.H.',
   'Saunders',
   '(',
   'St.',
   'Lawrence',
   'River',
   ')',
   '(',
   '968',
   'MW',
   ')'],
  [';', "'", "''", 'Anders', 'Lindström', "''", "'"]],
 'ner_tags': [[3, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0], [0, 0, 0, 1, 2, 0, 0]],
 'langs': [['en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en'],
  ['en', 'en', 'en', 'en', 'en', 'en', 'en']],
 'spans': [['ORG: R.H. Saunders', 'ORG: St. Lawrence River'],
  ['PER: Anders Lindström']]}

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
from transformers import BertModel
bert = BertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import BertTokenizer,BertForTokenClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer.add_tokens(['B_geo','I_geo','B_per','I_per','B_org','I_org'])

6

# New Section

In [ ]:
data = pd.read_csv('file1.csv')
data.head()

,Unnamed: 0,text,labels
0,0,The address of Persint is 6750 Koskikatu 25 Ap...,O O O B-ORGANIZATION O B-STREET_ADDRESS I-STRE...
1,1,What are my options?,O O O O
2,2,Krisztián Szöllösy listed his top 20 songs for...,B-PERSON I-PERSON O O O O O O O O O O O O O O ...
3,3,The Exversion Orchestra was founded in 1977. S...,O B-ORGANIZATION O O O O B-DATE_TIME O O O O O...
4,4,My name is Rubija,O O O B-PERSON


In [ ]:
def preprocess_dataset(data): 
    '''
        Here we will remove all the tags except for 'Org','Geo', and 'Per'
        type. These three are our targeted Entities.
    '''
#     data['labels'] = data['labels'].str.replace('-','_')
    data['labels'] = data['labels'].str.replace('B_gpe','O')
    data['labels'] = data['labels'].str.replace('I_gpe','O')
    data['labels'] = data['labels'].str.replace('B_tim','O')
    data['labels'] = data['labels'].str.replace('I_tim','O')
    data['labels'] = data['labels'].str.replace('B_eve','O')
    data['labels'] = data['labels'].str.replace('I_eve','O')
    data['labels'] = data['labels'].str.replace('B_nat','O')
    data['labels'] = data['labels'].str.replace('I_nat','O')
    data['labels'] = data['labels'].str.replace('B_art','O')
    data['labels'] = data['labels'].str.replace('I_art','O')
    
    return data

In [ ]:
data=preprocess_dataset(data)

In [ ]:
'''
This is to remove the sentences that doesn't contain our targeted entities.
'''
sum=0
for index, i in enumerate(data['labels']):
    a=set(i.split(' '))
    if(len(a)<=1):
        data.drop(labels=index, axis=0,inplace=True)
        sum+=1    
print(sum)
data = data.dropna()
data.reset_index(drop=True, inplace=True)

339


In [ ]:
unique_tags = data.labels.apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0)
unique_tags

I-STREET_ADDRESS        5448.0
O                      43101.0
B-ORGANIZATION           663.0
B-STREET_ADDRESS        1548.0
I-PERSON                1530.0
B-PERSON                2541.0
B-DATE_TIME              357.0
B-GPE                   1137.0
B-CREDIT_CARD            408.0
B-US_SSN                  48.0
B-US_DRIVER_LICENSE       15.0
B-AGE                    222.0
B-ZIP_CODE               111.0
B-TITLE                  216.0
I-ORGANIZATION           615.0
B-DOMAIN_NAME             72.0
B-EMAIL_ADDRESS          108.0
B-PHONE_NUMBER           240.0
I-DATE_TIME               60.0
I-GPE                    291.0
I-TITLE                  126.0
I-PHONE_NUMBER           429.0
B-IBAN_CODE               63.0
B-IP_ADDRESS              42.0
B-NRP                    165.0
I-ZIP_CODE                15.0
dtype: float64

In [ ]:
label_to_ids = {'B-geo': 1,
 'B-org': 2,
 'B-per': 3,
 'I-geo': 4,
 'I-org': 5,
 'I-per': 6,
 'O': 0}

In [ ]:
ids_to_label = {1:'B-geo',
 2:'B-org',
 3:'B-per',
 4:'I-geo',
 5:'I-org',
 6:'I-per',
 0:'O'}

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):

    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []
    
#     print('##')
#     print(sentence)
#     print(text_labels)
#     print('###')
    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)
        
        ## if sentence consist of more than 125 words, discard the later words.
        if(len(tokenized_sentence)>=125):
            return tokenized_sentence, labels
        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [ ]:
class Ner_Data(Dataset):

    def __init__(self, data):
        self.data = data
#         print("dataloader initialized")
        
        

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
#         print(idx)
        sentence = self.data['text'][idx].strip().split()  
        word_labels = self.data['labels'][idx].split(" ") 
#         print(len(sentence))
#         if(len(sentence)>64):
#             sentence=sentence[:63]
#             word_labels=word_labels[:63]
# #         print(sentence)
        t_sen, t_labl = tokenize_and_preserve_labels(sentence, word_labels, tokenizer)
                
        sen_code = tokenizer.encode_plus(t_sen,    
            add_special_tokens=True,  # Add [CLS] and [SEP]
            max_length = 128,  # maximum length of a sentence
            pad_to_max_length=True,  # Add [PAD]s
            return_attention_mask = True,  # Generate the attention mask
#             return_tensors = 'pt'
            )
             
            
        labels = [-100]*128
        for i, tok in enumerate(t_labl):
#             tok = tokenizer.convert_ids_to_tokens(i)
#             print(tok)
#             print(tok)
#             print(label_to_ids.get(tok))
            if label_to_ids.get(tok) != None:
                labels[i+1]=label_to_ids.get(tok)

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in sen_code.items()}
        item['labels'] = torch.as_tensor(labels)

        return item

In [ ]:
train_data = Ner_Data(data)

In [ ]:
print(len(train_data[10]['input_ids']))
print(len(train_data[10]['labels']))
print(train_data[10]['labels'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


128
128
tensor([-100,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0, -100,    0,    0,    0,    3,    0,    3,    3,    6,    6,    0,
           0,    0,    0,    0, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100])


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
train_data[148]
print(train_data[148]['input_ids'].detach().numpy())
print(tokenizer.convert_ids_to_tokens(train_data[148]['input_ids'].detach().numpy()))

print('#####')
for i in train_data[148]['labels'].detach().numpy():
#     print(i)
    print(ids_to_label.get(i))

[  101  2739  4311  2405  4465  1999  1996  2142  2163  2360  1996 15727
  2015  1010  2112  1997  2019  4866  2981  3189  2000  2022  2207  2006
  6928  1010  6232  4697  1037  2280  2327 14895  2000  3187  1011  2236
 12849  8873  4698  2078  1010  3841  2239  7367  6212  1010  1998  1996
  1057  1012  1050  1012  2436  2002  3753  2008  2743  1996  3514  1011
  2005  1011  2833  2565  1012   102     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
['[CLS]', 'news', 'reports', 'published', 'sunday', 'in', 'the', 'united', 'states', 'say', 'the', 'audit', '##s', ',', 'part', 'of', 'an', 'extensive', 'independent', 'report', 'to', 'be', 'released', 'on', 'monday', ',

In [ ]:
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=False)

In [ ]:
len(train_data)

40917

In [ ]:
model2 = model =BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(label_to_ids))
model2.to(device)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
learning_rate = 0.0001
batch_size = 64
epochs = 5

In [ ]:
loss_fn2 = nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def train_loop(train_dataloader, model, optimizer):
    size = len(train_dataloader.dataset)
    train_loss =0
    for i,sample in enumerate(train_dataloader):
        optimizer.zero_grad()
#         print(sample)
        ids=sample['input_ids'].to(device)
        mask=sample['attention_mask'].to(device)
        labels = sample['labels'].to(device)
        pred = model2(input_ids=ids, attention_mask=mask ,labels = labels )
        loss = pred[0]
        
#         print(f"loss: {loss.item()}")
        train_loss+=loss.item()
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        
        if(i>0 and i % 500==0):
            print(f"loss: {train_loss/i:>4f}  [{i:>5d}/{size/32}]")
    return train_loss

In [ ]:
epochs = 6
train_loss = []
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    loss = train_loop(train_dataloader, model, optimizer)
    train_loss.append(loss)
#     test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
